1- API Key Loading

In [ ]:
import os
from dotenv import load_dotenv

# The "../" tells Python to look one folder up (in the WEEK 1 root)
load_dotenv("../.env")

# Retrieve the key
api_key = os.getenv("OPENAI_API_KEY")

if api_key:
    print("✅ API Key successfully loaded!")
else:
    print("❌ API Key not found. Check your .env file location.")

2. Imports, Data Loading, and Initial Setup

In [ ]:
from datasets import load_dataset
import pandas as pd
from pathlib import Path
from PIL import Image
import base64

# Load the data
dataset = load_dataset("ashraq/fashion-product-images-small", split="train[:100]")
products_df = pd.DataFrame(dataset)

# Add a dummy 'price' column if it doesn't exist
if 'price' not in products_df.columns:
    products_df['price'] = 29.99 # Assign a default price, adjust as needed

#save the images to a local folder    

images_dir = Path("product_images")
images_dir.mkdir(exist_ok=True)
print("✓ Data loaded and folder created.")

# Save each image to the local folder
for index, row in products_df.iterrows():
    image = row['image']
    image.save(images_dir / f"{row['id']}.jpg")

    # Save the PIL image object as a physical file
    if isinstance(row['image'], Image.Image):
            image_path = images_dir / f"{row['id']}.jpg"
            row['image'].save(image_path)           

print(f"✓ Saved images to {images_dir}")

In [ ]:
def encode_image(image_path):
    """Convert a physical image file to a base64 string."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

5. create_product_listing_prompt Function Definition

In [ ]:
def create_product_listing_prompt(product_name, price, category, additional_info=None):
    """
    Create a prompt for generating product listings.
    
    Parameters:
    - product_name: Name of the product
    - price: Price of the product
    - category: Product category
    - additional_info: Optional additional information
    
    Returns:
    - Formatted prompt string
    """
    prompt = f"""You are an expert e-commerce copywriter. Analyze the product image and create a compelling product listing.

Product Information:
- Name: {product_name}
- Price: ${price:.2f}
- Category: {category}
{f'- Additional Info: {additional_info}' if additional_info else ''}

Please create a professional product listing that includes:

1. **Product Title** (catchy, SEO-friendly, 60 characters max)
2. **Product Description** (detailed, 150-200 words)
   - Highlight key features and benefits
   - Use persuasive language
   - Include relevant details visible in the image
3. **Key Features** (bullet points, 5-7 items)
4. **SEO Keywords** (comma-separated, 10-15 relevant keywords)

Format your response as JSON with the following structure:
{{
    "title": "Product title here",
    "description": "Full description here",
    "features": ["Feature 1", "Feature 2", ...],
    "keywords": "keyword1, keyword2, ..."
}}

Be specific about what you see in the image. Mention colors, materials, design elements, and any distinctive features."""
    
    return prompt

# Test prompt creation
test_prompt = create_product_listing_prompt(
    product_name="Wireless Bluetooth Headphones",
    price=79.99,
    category="Electronics",
    additional_info="Noise cancelling, 30-hour battery"
)

print("\n" + "="*50)
print("PROMPT TEMPLATE")
print("="*50)
print(test_prompt[:1000] + "...")  # Show first 500 characters


Prepare the API request with image and prompt

In [ ]:
import os
import json
import base64
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv("../.env")
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

# 1. Function to encode the image to base64 (Required for Step 3/5)
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# 2. Select a product to test (First product in your DataFrame)
product_data = products_df.iloc[0]
image_path = images_dir / f"{product_data['id']}.jpg"

# 3. Generate the prompt using your Step 4 function
prompt_text = create_product_listing_prompt(
    product_name=product_data.get('productDisplayName', 'Product'),
    price=product_data['price'],
    category=product_data.get('masterCategory', 'General')
)

# 4. Prepare the API Call
print(f"Calling OpenAI for product: {product_data.get('productDisplayName')}...")

try:
    # We use 'gpt-4o' or 'gpt-4-turbo' because they have Vision capabilities
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt_text},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{encode_image(image_path)}"
                        },
                    },
                ],
            }
        ],
        response_format={ "type": "json_object" } # Forces the AI to give us valid JSON
    )

    # 5. Parse the Response
    result_text = response.choices[0].message.content
    listing_json = json.loads(result_text)

    print("\n✓ API Response Received!")
    print(json.dumps(listing_json, indent=4))

except Exception as e:
    print(f"⚠ Error calling the API: {e}")

In [ ]:
import requests  
import time      
import json
import os

# 1. Ensure the key is loaded
api_key = os.getenv("OPENAI_API_KEY")

all_product_listings = []
API_ENDPOINT = "https://api.openai.com/v1/chat/completions"

for index, product_data in products_df.head(5).iterrows():
    print(f"\n--- Processing Product ID: {product_data['id']} ---")
    try:
        # Preparation
        image_path = images_dir / f"{product_data['id']}.jpg"
        if not image_path.exists():
            continue

        base64_image = encode_image(image_path)
        product_prompt = create_product_listing_prompt(
            product_name=product_data['productDisplayName'],
            price=product_data['price'],
            category=product_data['masterCategory']
        )

        # 2. Build the request
        payload = {
            "model": "gpt-4o-mini",
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": product_prompt},
                        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                    ]
                }
            ],
            "response_format": { "type": "json_object" } # Added this for easier parsing
        }

        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {api_key}"
        }

        # 3. Execution
        response = requests.post(API_ENDPOINT, headers=headers, json=payload)
        response.raise_for_status() 

        # 4. Parsing
        result_text = response.json()['choices'][0]['message']['content']
        product_listing = json.loads(result_text)
        
        all_product_listings.append({
            "product_id": product_data['id'],
            "listing": product_listing
        })
        print(f"✓ Success for ID: {product_data['id']}")

    except Exception as e:
        print(f"⚠ Failed for ID {product_data['id']}: {e}")

    time.sleep(1) 

print(f"\nDone! Processed {len(all_product_listings)} products.")


In [ ]:
# Verify the results are saved correctly in the list
print(f"Items in list: {len(all_product_listings)}")

for item in all_product_listings:
    pid = item['product_id']
    title = item['listing'].get('title', 'No Title')
    print(f"ID {pid}: {title}")